### Simple EDA

간단하게 데이터를 탐색한다.

In [83]:
import re

test = 'a[]'
for i in test:
    print(i)
    re.match('[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]', i)

a
[
]


In [5]:
import time
from selenium import webdriver
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
import re

# f=open('./data/renewal_dataset.csv','w',encoding="utf-8",newline='')
# wr=csv.writer(f)
# wr.writerow(['title', 'content', 'views'])
# f.close()

options = webdriver.ChromeOptions()
options.add_argument("headless") # 창 숨기고 실행
options.add_argument('--start-maximized') # 전체화면 시작
options.add_argument('--disable-gpu') # gpu 비활성화
options.add_experimental_option('excludeSwitches', ['enable-logging']) # 불필요한 로그 메세지 출력 x
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36") # user-agent 추가

# 네이버 로그인 url
url = 'https://nid.naver.com/nidlogin.login'
id="gjsdn245"
pw="1831reatqieq"

browser=webdriver.Chrome(options = options) # options = options
browser.get(url)

browser.implicitly_wait(2)

# 로그인
browser.execute_script("document.getElementsByName('id')[0].value=\'"+ id + "\'")
browser.execute_script("document.getElementsByName('pw')[0].value=\'"+ pw + "\'")

browser.find_element(by=By.XPATH,value='//*[@id="log.login"]').click()
time.sleep(1)

headers = {'user-agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}

baseurl = ['https://cafe.naver.com/cheepboracay', "https://cafe.naver.com/freedogcat"]
clubid = [20720299, 14435163]
article_id = [628372, 888606]
idx = 0
all_data = []
article_decrease = 0
for i in range(len(baseurl)):
    for article_decrease in range(article_id[i]):
        time.sleep(1)
        article_decrease += 1
        row_data = []
        browser.get(baseurl[i])
        
        idx += 1
        time.sleep(1)
        browser.get(f"{baseurl[i]}/ArticleRead.nhn?clubid={clubid[i]}&menuid=2&boardtype=L&articleid={article_id[i]}&referrerAllArticles=false")
        time.sleep(1)
        article_id[i] -= article_decrease
        
        try:
            main = browser.find_element(By.ID, "main-area")
        except:
            print('삭제된 게시글입니다.')
            continue
        
        time.sleep(1)
        browser.switch_to.frame('cafe_main') 
        # article = main.find_element(By.TAG_NAME, "iframe")
        # src = article.get_attribute("src")
        # time.sleep(1)
        # browser.get(src)
        # time.sleep(1)
        soup = bs(browser.page_source, "html.parser")
        try:
            main2 = soup.find("div", {"id":"main-area"})
            title = main2.find("div", {"class":"tit-box"})
            title_text = [i for i in title.text.split("\n") if i != ''][0]
            title_text = re.sub('[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|a-zA-Z ]', '', title_text)
            
            
            if '[' in title_text and ']' in title_text:
                regex = title_text[title_text.find("[") : title_text.find("]") + 1]
                title_text = title_text.replace(regex, "")

            contents = main2.find("div", {"id":"tbody"})
            contents_text = "".join(contents.text.split("  "))
            contents_text = re.sub('[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|a-zA-Z ]', '', contents_text)

            views_box = main2.find("div", {"id":"cmtMenu"}).text.split("\n")
            views = [x for x in views_box if "조회수" in x][0]
            views_cnt = views[views.find(" ") + 1:]
            
            print([title_text, contents_text, views_cnt])
            
            f = open(f'./data/renewal_dataset.csv', 'a+', newline='', encoding = "utf-8")
            wr = csv.writer(f)
            wr.writerow([title_text, contents_text, views_cnt])
            f.close()
        except:
            print("연결할 수 없거나, 카페 회원에게만 공개된 게시글입니다.")

['오랜만에 출첵', '저번주에 일본가서 얼굴꽝하고ㅠㅠ돌아왔어요 ㅋㅋ월에 시부모님 모시고 보라가요 어후 가서 더 조심해야짓 기다려랏  ', '58']
['헤난 파크  헤난 팜비치 견적 문의합니다', '예약가능한 보홀 리조트 사우스팜블루워터보홀쇼어 헤난알로나타왈라 예약가능한 보라카이 리조트 샹그릴라크림슨모벤픽아샤아쿠아펠리즈아잘리아 헤난크리스탈라군가든팜프라임리젠시파크                            정확한 견적은 정확한 정보로부터 나옵니다번거로우시더라도 양식에 맞게 기입해주세요  성인의 기준은 리조트마다 다릅니다 자녀분이있다면 만나이도 기입해주세요  리조트 견적 게시판은 원하시는 리조트의 요금을 알려드리는 곳으로써최종가능여부는 실제예약인 리조트예약 게시판으로 신청시 알수있는점 참고바랍니다 리조트에 관한 불만사항은 반드시 현장에서 리조트 측에 문의를 주셔야합니다 리조트의 경우에는 리조트 정책상 어떠한 경우에도 환불 및 변경 취소가 불가능한 조건이므로 신중하게 선택하세요                            리조트 견적   견적을 원하시는 지역  보라카이  리조트 체크인 날짜  년 월 일리조트 체크아웃 날짜  년 월 일   리조트 이름 헤난 파크 헤난 팜비치 룸타입 디럭스룸  총 숙박기간 박 일  성인인원아이인원만 나이 기재필수 성인명 아동명 만세  ', '53']
연결할 수 없거나, 카페 회원에게만 공개된 게시글입니다.
['투어픽샌견적', ' 견적받길 원하는 항목 투어 O  픽업샌딩 O   항공정보 비행편명입국편명출국편명   예시  PRPR 한국에서 출발 날짜시간예시 년 월 일 오전 분 출발 칼리보공항 출발 날짜시간  예시 년 월 일 오후 분 출발   리조트 정보 체크인 리조트 예시 헤난 크리스탈 샌즈 리조트 체크아웃 리조트  예시 크림슨 리조트  총 인원  명 남녀  원하는 투어마사지  픽업샌딩써니요트호핑아일랜드투어말룸파티 파밀리야 드라이마사지시간 추가하게되면 비용이 얼마가 추가될까요 반드시 읽어주세요 픽업샌딩 왕복에 필요한 모든 비용포

KeyboardInterrupt: 

In [1]:
import pandas as pd

# 컬럼이 없는 데이터 프레임들은 임의로 컬럼명을 생성하여 사용한다.
df1 = pd.DataFrame(pd.read_csv("./crawl_14435163.csv").values, columns=['title', 'write_date', 'views']) # 데이터프레임 1
df2 = pd.DataFrame(pd.read_csv("./crawl_20720299.csv").values, columns=['title', 'write_date', 'views']) # 데이터프레임 2

df1 = df1[['title', 'views']] # title과 views열만 사용한다
df2 = df2[['title', 'views']]
df3 = pd.read_csv("./naver_cafe_crawl_fix.csv")[['title', 'views']]

### about df1

In [2]:
df1.isna().sum() # 결측치 확인

title    10
views    10
dtype: int64

In [3]:
df1.info() # data type 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75059 entries, 0 to 75058
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   75049 non-null  object
 1   views   75049 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [4]:
df1.dropna(inplace=True, axis = 0) # 결측치 제거

df1['title'] = df1['title'].str.replace("[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|a-zA-Z ]", "", regex=True) # 한글과 알파벳, 공백을 제외하고 제거
df1['views'] = df1['views'].str.replace("[^0-9]", "", regex=True).astype(int) # label값을 구하기 위해 int타입으로 형전환

dup_cnt = df1['title'].loc[df1['title'].duplicated() == True].count() # 중복되는 개수 조회
print(f"{dup_cnt}개의 중복 자료가 존재합니다.")

df1.drop_duplicates(subset=['title'], inplace=True) # 중복 행 제거
df1 = df1.reset_index(drop = True) # 인덱스 초기화
print(f"{df1.isna().sum().sum()}개의 결측치가 존재합니다.") # 결측치 확인

57456개의 중복 자료가 존재합니다.
0개의 결측치가 존재합니다.


In [5]:
df1.head() # 한글과 알파엣, 공백이 잘 지워졌는지 확인

,title,views
0,분양중고양이무료분양 스코티쉬코숏 믹스 살 여아 바다의 새로운 가족이 되어주세요경남 김해,18
1,무엇이든 물어보살,16
2,서마일,9
3,안아드리오,6
4,안녕하세요,1


### about df2

In [6]:
df2.isna().sum() # 결측치 확인

title    10242
views    10242
dtype: int64

In [7]:
df2.info() # data type 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85341 entries, 0 to 85340
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   75099 non-null  object
 1   views   75099 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [8]:
# df1과 동일한 방식으로 진행
df2.dropna(inplace=True, axis = 0)

df2['title'] = df2['title'].str.replace("[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|a-zA-Z ]", "", regex=True)
df2['views'] = df2['views'].str.replace("[^0-9]", "", regex=True).astype(int)

dup_cnt = df2['title'].loc[df2['title'].duplicated() == True].count()
print(f"{dup_cnt}개의 중복 자료가 존재합니다.")

df2.drop_duplicates(subset=['title'], inplace=True)
df2 = df2.reset_index(drop = True)
print(f"{df2.isna().sum().sum()}개의 결측치가 존재합니다.")

62989개의 중복 자료가 존재합니다.
0개의 결측치가 존재합니다.


### about df3

In [9]:
df3.isna().sum() # 결측치 확인

title    0
views    1
dtype: int64

In [10]:
df3.info() # data type 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43168 entries, 0 to 43167
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   43168 non-null  object 
 1   views   43167 non-null  float64
dtypes: float64(1), object(1)
memory usage: 674.6+ KB


In [11]:
# 소수점 자리 아래 확인을 위해서 views 컬럼을 string으로 타입캐스팅
df3['views'] = df3['views'].astype('string')
df3['views'].loc[df3['views'].str.endswith(".0") == True].count() # 모두 소수점 아래 자리는 0이다.

43167

In [12]:
df3.dropna(inplace=True, axis = 0) # 결측치 제거

In [13]:
# 소수점 아래 자리가 모두 .0 이므로 차후에 정수형으로 변환한다.
df3['views'] = df3['views'].astype('string')
df3['views'][0].endswith(".0")

True

In [14]:
#  df1, df2에서 진행한 전처리과정 동일하게 진행
df3['title'] = df3['title'].str.replace("[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|a-zA-Z ]", "", regex=True)
df3['views'] = df3['views'].str.replace("[^0-9].", "", regex=True).astype(float).astype(int)

dup_cnt = df3['title'].loc[df3['title'].duplicated() == True].count()
print(f"{dup_cnt}개의 중복 자료가 존재합니다.")

df3.drop_duplicates(subset=['title'], inplace=True)
df3 = df3.reset_index(drop = True)
print(f"{df3.isna().sum().sum()}개의 결측치가 존재합니다.")

669개의 중복 자료가 존재합니다.
0개의 결측치가 존재합니다.


In [15]:
df = pd.concat([df1, df2, df3]).reset_index(drop = True) # 전처리 완료한 세 개의 데이터 프레임을 합친다.

In [16]:
df.isna().sum() # 결측치 다시확인

title    0
views    0
dtype: int64

In [17]:
df.head()

,title,views
0,분양중고양이무료분양 스코티쉬코숏 믹스 살 여아 바다의 새로운 가족이 되어주세요경남 김해,18
1,무엇이든 물어보살,16
2,서마일,9
3,안아드리오,6
4,안녕하세요,1


In [18]:
df.info() # data type 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72201 entries, 0 to 72200
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   72201 non-null  object
 1   views   72201 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 846.2+ KB


### 라벨링 진행
각 view_cnt의 평균을 내서 평균보다 높으면 1, 아니라면 0을 부여한다.

In [19]:
mean_val = int(df['views'].mean())

In [20]:
mean_val

250

In [21]:
def get_label_list(values, standard):
    label_list = []
    for value in values:
        if value >= standard:
            label_list.append(1)
        else:
            label_list.append(0)
            
    return label_list

df['label'] = get_label_list(df['views'].values, mean_val)

### 데이터 병합
전처리 후 라벨링 한 두 데이터 프레임을 병합합니다.

In [22]:
df

,title,views,label
0,분양중고양이무료분양 스코티쉬코숏 믹스 살 여아 바다의 새로운 가족이 되어주세요경남 김해,18,0
1,무엇이든 물어보살,16,0
2,서마일,9,0
3,안아드리오,6,0
4,안녕하세요,1,0
...,...,...,...
72196,편백루바로 만든 장난감 강아지집,249,0
72197,목공은 너무 재밌는거 같아요,467,1
72198,박스훼손품 그 이후,336,1
72199,사는이야기미세먼지냐 추위냐네요,46,0


### 라벨값 분포 확인
각 라벨별 분포를 확인합니다.

In [23]:
df['label'].value_counts()

label
0    46544
1    25657
Name: count, dtype: int64

### 시행착오 1 : 라벨값 간 불균형

평균

|라벨 0|라벨 1|
|---|---|
|46544건|25657건|

평균은 라벨값 간의 큰 격차를 불러오기에, 분위수를 사용한다

제 1 사분위수

|라벨 0|라벨 1|
|---|---|
|18040건|54161건|

제 2 사분위수

|라벨 0|라벨 1|
|---|---|
|36001건|36200건|

그나마 균등하게 분포되는 제 2 사분위수로 채택

In [24]:
df_quantile1 = df['views'].quantile(.5)
print(int(df_quantile1))

176


In [25]:
df['label'] = get_label_list(df['views'], df_quantile1)

In [26]:
df

,title,views,label
0,분양중고양이무료분양 스코티쉬코숏 믹스 살 여아 바다의 새로운 가족이 되어주세요경남 김해,18,0
1,무엇이든 물어보살,16,0
2,서마일,9,0
3,안아드리오,6,0
4,안녕하세요,1,0
...,...,...,...
72196,편백루바로 만든 장난감 강아지집,249,1
72197,목공은 너무 재밌는거 같아요,467,1
72198,박스훼손품 그 이후,336,1
72199,사는이야기미세먼지냐 추위냐네요,46,0


In [29]:
df['label'].value_counts()
# df.to_csv("./origin_df.csv", sep=",", encoding="utf8", index=False)

label
1    36200
0    36001
Name: count, dtype: int64

In [32]:
df['views'].mean()

250.00933505076108

In [28]:
pd.read_csv("./origin_df.csv")

,title,views,label
0,분양중고양이무료분양 스코티쉬코숏 믹스 살 여아 바다의 새로운 가족이 되어주세요경남 김해,18,0
1,무엇이든 물어보살,16,0
2,서마일,9,0
3,안아드리오,6,0
4,안녕하세요,1,0
...,...,...,...
72196,편백루바로 만든 장난감 강아지집,249,1
72197,목공은 너무 재밌는거 같아요,467,1
72198,박스훼손품 그 이후,336,1
72199,사는이야기미세먼지냐 추위냐네요,46,0
